In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from utils import *
import datetime
# Setup the random number of external data
# The portion needs to be between 0 and 1 and requires sensitivity analysis
seed = 2
np.random.seed(seed=seed)
portion_external = 0.4 #np.round(np.random.rand(),1)
print(f"portion_external: {portion_external}")


portion_external: 0.4


# AMLSIM Bank Mixed data

In [2]:
# Load the AML bank_mixed dataset
bank_mixed_accounts = pd.read_csv("data/AMLSim/sample_data/banks/data/bank_mixed/accounts.csv")
bank_mixed_transactions = pd.read_csv("data/AMLSim/sample_data/banks/data/bank_mixed/transactions.csv")
bank_mixed_alert_txn = pd.read_csv("data/AMLSim/sample_data/banks/data/bank_mixed/alert_transactions.csv")
bank_mixed_features = pd.read_csv("data/AMLSim/sample_data/banks/features/bank_mixed/features.csv")

In [3]:
# See the shape of the data
print(f"bank_mixed_accounts: {bank_mixed_accounts.shape}")
print(f"bank_mixed_transactions: {bank_mixed_transactions.shape}")
print(f"bank_mixed_alert_txn: {bank_mixed_alert_txn.shape}")
print(f"bank_mixed_features: {bank_mixed_features.shape}")

bank_mixed_accounts: (20000, 24)
bank_mixed_transactions: (885744, 8)
bank_mixed_alert_txn: (807, 9)
bank_mixed_features: (20000, 74)


In [94]:
bank_mixed_accounts.head()

,acct_id,dsply_nm,type,acct_stat,acct_rptng_crncy,prior_sar_count,branch_id,open_dt,close_dt,initial_deposit,...,street_addr,city,state,country,zip,gender,birth_date,ssn,lon,lat
0,0,C_0,SAV,A,USD,False,1,2017-01-01T00:00:00Z,4754-11-29T00:00:00Z,92221.09,...,NaN,NaN,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN
1,1,C_1,SAV,A,USD,False,1,2017-01-01T00:00:00Z,4754-11-29T00:00:00Z,87897.72,...,NaN,NaN,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN
2,2,C_2,SAV,A,USD,False,1,2017-01-01T00:00:00Z,4754-11-29T00:00:00Z,71028.58,...,NaN,NaN,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN
3,3,C_3,SAV,A,USD,False,1,2017-01-01T00:00:00Z,4754-11-29T00:00:00Z,62945.84,...,NaN,NaN,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN
4,4,C_4,SAV,A,USD,False,1,2017-01-01T00:00:00Z,4754-11-29T00:00:00Z,75563.74,...,NaN,NaN,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# How many SAR accounts are there?
bank_mixed_accounts['prior_sar_count'].sum(),bank_mixed_accounts['prior_sar_count'].sum()/len(bank_mixed_accounts['prior_sar_count'])

(753, 0.03765)

How does the account SAR related with transaction ones?

In [5]:
print(bank_mixed_alert_txn['bene_acct'].unique().shape)

bank_mixed_accounts['label'] = bank_mixed_accounts['prior_sar_count'].copy()
bank_mixed_accounts['label'].loc[bank_mixed_alert_txn['bene_acct'].unique()] = True
bank_mixed_accounts['label'].sum()

(530,)


/tmp/ipykernel_3392278/1066052114.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank_mixed_accounts['label'].loc[bank_mixed_alert_txn['bene_acct'].unique()] = True


753

### Process the features

In [6]:
bank_mixed_features

,account,count,sum_amount,in_count,sum_in_amount,max_in_amount,avg_in_amount,out_count,sum_out_amount,max_out_amount,...,gs_date_ordered,gs_in_count,gs_out_count,gs_in_amount,gs_out_amount,gs_acct_amount_diff,gs_acct_amount_ratio,gs_acct_period,gs_acct_date_diff,gs_depth
0,0,58,17751.04,0,0.00,0.00,0.000000,58,17751.04,2698.29,...,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.0
1,1,57,12714.19,6,312.65,54.96,52.108333,51,12401.54,2651.81,...,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.0
2,2,27,3814.59,1,48.78,48.78,48.780000,26,3765.81,2515.04,...,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.0
3,3,60,15142.47,4,192.78,51.28,48.195000,56,14949.69,2634.38,...,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.0
4,4,66,8155.62,13,640.32,54.91,49.255385,53,7515.30,2584.59,...,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,342,95650.54,293,81403.57,2749.66,277.827884,49,14246.97,2560.93,...,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.0
19996,19996,335,90026.46,286,77568.00,2731.21,271.216783,49,12458.46,2654.16,...,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.0
19997,19997,366,116141.57,316,96346.59,2729.76,304.894272,50,19794.98,2738.71,...,1.0,3.0,5.0,138.64,240.88,-102.24,0.575556,617.0,607.0,0.5
19998,19998,460,150296.98,412,138572.86,2745.54,336.341893,48,11724.12,2697.92,...,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.0


In [98]:
bank_mixed_features.loc[1]

account                     1.000000
count                      57.000000
sum_amount              12714.190000
in_count                    6.000000
sum_in_amount             312.650000
max_in_amount              54.960000
avg_in_amount              52.108333
out_count                  51.000000
sum_out_amount          12401.540000
max_out_amount           2651.810000
avg_out_amount            243.167451
min_interval                2.000000
avg_interval               12.625000
max_interval               51.000000
ego1_num                    0.000000
ego1_prop                   0.000000
ego2_num                    0.000000
ego2_prop                   0.000000
pr                          0.000012
u_sar                       0.000000
u_length_min                0.000000
u_length_avg                0.000000
u_length_max                0.000000
u_amount_min                0.000000
u_amount_avg                0.000000
u_amount_max                0.000000
u_period_min                0.000000
u

In [99]:
# No missing values in the features
pd.isna(bank_mixed_features).any().any()

False

In [100]:
bank_mixed_features.describe()

,account,count,sum_amount,in_count,sum_in_amount,max_in_amount,avg_in_amount,out_count,sum_out_amount,max_out_amount,...,gs_date_ordered,gs_in_count,gs_out_count,gs_in_amount,gs_out_amount,gs_acct_amount_diff,gs_acct_amount_ratio,gs_acct_period,gs_acct_date_diff,gs_depth
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000,20000.000000,20000.00000,...,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,9999.500000,88.574400,25982.214539,44.287200,12991.107270,2430.472821,292.630436,44.28720,12991.107270,2583.54187,...,0.084700,0.517123,0.524457,159.784872,162.409194,-2.624322,0.218433,46.202700,5.433992,0.035122
std,5773.647028,43.092613,14473.309948,41.281889,13108.189261,693.280974,143.794757,8.71894,5575.372476,404.05466,...,0.278442,4.621628,2.909527,1425.405561,1027.650829,1412.813813,4.170396,162.139742,57.503636,0.176598
min,0.000000,7.000000,361.730000,0.000000,0.000000,0.000000,0.000000,3.00000,293.810000,53.73000,...,0.000000,0.000000,0.000000,0.000000,0.000000,-31366.920000,0.000000,0.000000,0.000000,0.000000
25%,4999.750000,66.000000,17501.845000,23.000000,5814.747500,2528.742500,203.081138,39.00000,9284.012500,2589.52750,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,9999.500000,78.000000,23275.940000,34.000000,9523.505000,2652.950000,283.460326,45.00000,12340.555000,2671.70000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,14999.250000,96.000000,30735.290000,50.000000,16025.937500,2714.935000,368.531315,50.00000,16852.507500,2718.07000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,19999.000000,550.000000,178694.120000,507.000000,163042.910000,2996.940000,2746.390000,86.00000,36361.510000,2996.94000,...,1.000000,134.000000,71.000000,42089.030000,31366.920000,41844.480000,177.173906,719.000000,712.000000,1.000000


In [101]:
# See the transaction data
bank_mixed_transactions

,tran_id,orig_acct,bene_acct,tx_type,base_amt,tran_timestamp,is_sar,alert_id
0,1,5388,8992,TRANSFER,52.30,2017-01-01T00:00:00Z,False,-1
1,2,12024,14776,TRANSFER,51.06,2017-01-01T00:00:00Z,False,-1
2,3,2024,3834,TRANSFER,105.63,2017-01-01T00:00:00Z,False,-1
3,4,15550,8603,TRANSFER,46.75,2017-01-01T00:00:00Z,False,-1
4,5,5365,17449,TRANSFER,46.28,2017-01-01T00:00:00Z,False,-1
...,...,...,...,...,...,...,...,...
885739,885740,8908,13092,TRANSFER,49.49,2018-12-21T00:00:00Z,False,-1
885740,885741,4764,18898,TRANSFER,48.56,2018-12-21T00:00:00Z,False,-1
885741,885742,18145,6760,TRANSFER,54.92,2018-12-21T00:00:00Z,False,-1
885742,885743,12663,16891,TRANSFER,48.55,2018-12-21T00:00:00Z,False,-1


In [102]:
bank_mixed_accounts

,acct_id,dsply_nm,type,acct_stat,acct_rptng_crncy,prior_sar_count,branch_id,open_dt,close_dt,initial_deposit,...,city,state,country,zip,gender,birth_date,ssn,lon,lat,label
0,0,C_0,SAV,A,USD,False,1,2017-01-01T00:00:00Z,4754-11-29T00:00:00Z,92221.09,...,NaN,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN,False
1,1,C_1,SAV,A,USD,False,1,2017-01-01T00:00:00Z,4754-11-29T00:00:00Z,87897.72,...,NaN,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN,False
2,2,C_2,SAV,A,USD,False,1,2017-01-01T00:00:00Z,4754-11-29T00:00:00Z,71028.58,...,NaN,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN,False
3,3,C_3,SAV,A,USD,False,1,2017-01-01T00:00:00Z,4754-11-29T00:00:00Z,62945.84,...,NaN,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN,False
4,4,C_4,SAV,A,USD,False,1,2017-01-01T00:00:00Z,4754-11-29T00:00:00Z,75563.74,...,NaN,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,C_19995,SAV,A,USD,False,1,2017-01-01T00:00:00Z,4754-11-29T00:00:00Z,55164.17,...,NaN,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN,False
19996,19996,C_19996,SAV,A,USD,False,1,2017-01-01T00:00:00Z,4754-11-29T00:00:00Z,53700.46,...,NaN,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN,False
19997,19997,C_19997,SAV,A,USD,False,1,2017-01-01T00:00:00Z,4754-11-29T00:00:00Z,69771.38,...,NaN,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN,False
19998,19998,C_19998,SAV,A,USD,False,1,2017-01-01T00:00:00Z,4754-11-29T00:00:00Z,65446.37,...,NaN,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN,False


Generate the heterogeneous graph

In [7]:
# Generate the heterogenous data
bank_mixed_accounts_hetero = pd.DataFrame(columns=['account_id','internal','label'])
bank_mixed_features_hetero = bank_mixed_features.copy()
bank_mixed_features_hetero = bank_mixed_features_hetero.drop(columns=['account'])
# Probably consider drop the sg and gs features?

# Heteregenous accounts
bank_mixed_accounts_hetero['account_id'] = bank_mixed_accounts['acct_id']
bank_mixed_accounts_hetero['internal'] = np.random.rand(len(bank_mixed_features)) > portion_external
bank_mixed_accounts_hetero['label'] = bank_mixed_accounts['label'].astype(int)
bank_mixed_accounts_hetero['label'].loc[bank_mixed_accounts_hetero['internal']==0] = np.nan

# Transaction data
bank_mixed_transactions_hetero = pd.DataFrame(columns=['sender','receiver','txn_type'])
bank_mixed_transactions_hetero['sender'] = bank_mixed_transactions['orig_acct']
bank_mixed_transactions_hetero['receiver'] = bank_mixed_transactions['bene_acct']

bank_mixed_internal_accts = bank_mixed_accounts_hetero['account_id'][bank_mixed_accounts_hetero['internal']]
# Check the transaction type
transaction_senderflag = bank_mixed_transactions_hetero['sender'].isin(bank_mixed_internal_accts)
transaction_receiverflag = bank_mixed_transactions_hetero['receiver'].isin(bank_mixed_internal_accts)
bank_mixed_transactions_hetero['txn_type'] = transaction_senderflag.combine(transaction_receiverflag,check_edge_type)

# Drop the external -> external transactions
bank_mixed_transactions_hetero = bank_mixed_transactions_hetero[bank_mixed_transactions_hetero['txn_type'] != -1]

/tmp/ipykernel_3392278/226705668.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank_mixed_accounts_hetero['label'].loc[bank_mixed_accounts_hetero['internal']==0] = np.nan


In [104]:
# Save the data
bank_mixed_data_path = f'./hetero_data/amlsim_mixed/ext_{portion_external}/'
import os
if not os.path.exists(bank_mixed_data_path):
    os.makedirs(bank_mixed_data_path)

# Save to csv
bank_mixed_accounts_hetero.to_csv(bank_mixed_data_path + 'accounts.csv',index=False)
bank_mixed_transactions_hetero.to_csv(bank_mixed_data_path + 'transactions.csv',index=False)
bank_mixed_features_hetero.to_csv(bank_mixed_data_path + 'features.csv',index=False)

In [8]:
bank_mixed_accounts['label'].value_counts(normalize=True)

False    0.96235
True     0.03765
Name: label, dtype: float64